In [5]:
# dataset.py
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset


class ISICDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

        # ISIC 2019 class columns
        self.class_names = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC"]

        # Convert one-hot → class index
        self.df["label"] = self.df[self.class_names].values.argmax(axis=1)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]["image"]
        label = int(self.df.iloc[idx]["label"])

        img_path = os.path.join(self.image_dir, f"{img_id}.jpg")
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
# model.py
import torch.nn as nn
from torchvision import models


class DenseNetClassifier(nn.Module):
    def __init__(self, num_classes=7, freeze_backbone=True):
        super().__init__()

        self.model = models.densenet121(weights="IMAGENET1K_V1")

        if freeze_backbone:
            for param in self.model.features.parameters():
                param.requires_grad = False

        self.model.classifier = nn.Linear(
            self.model.classifier.in_features, num_classes
        )

    def forward(self, x):
        return self.model(x)

In [7]:
# trainer.py
import torch
from tqdm import tqdm


class Trainer:
    def __init__(self, model, device, optimizer, criterion):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion

    def train_one_epoch(self, loader):
        self.model.train()
        total_loss, correct, total = 0, 0, 0

        for images, labels in tqdm(loader, desc="Training"):
            images, labels = images.to(self.device), labels.to(self.device)

            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)

            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        return total_loss / len(loader), correct / total

    def validate(self, loader):
        self.model.eval()
        total_loss, correct, total = 0, 0, 0

        with torch.no_grad():
            for images, labels in tqdm(loader, desc="Validation"):
                images, labels = images.to(self.device), labels.to(self.device)

                outputs = self.model(images)
                loss = self.criterion(outputs, labels)

                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).sum().item()
                total += labels.size(0)

        return total_loss / len(loader), correct / total

In [8]:
# train.py
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn, optim


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(20),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    train_dataset = ISICDataset(
        csv_file="../data/slice/ISIC_SUBSET_5000/train.csv",
        image_dir="../data/slice/ISIC_SUBSET_5000/images",
        transform=transform,
    )

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    model = DenseNetClassifier(num_classes=7).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    trainer = Trainer(model, device, optimizer, criterion)

    epochs = 1
    best_loss = float("inf")
    for epoch in range(epochs):
        train_loss, train_acc = trainer.train_one_epoch(train_loader)

        print(
            f"Epoch [{epoch+1}/{epochs}] "
            f"Train Loss: {train_loss:.4f} "
            f"Train Acc: {train_acc:.4f}"
        )

        if train_loss < best_loss:
            best_loss = train_loss
            torch.save(
                {
                    "model_name": "densenet121",
                    "num_classes": 7,
                    "freeze_backbone": True,
                    "state_dict": model.state_dict(),
                },
                "densenet_isic.pth",
            )


main()

Training: 100%|██████████| 63/63 [05:49<00:00,  5.55s/it]

Epoch [1/1] Train Loss: 1.2383 Train Acc: 0.5493
